In [16]:
import pandas as pd
from datetime import datetime
import numpy as np
from google.cloud import bigquery
from bq_carrefour import MethodBQ

In [17]:
'''
Funcion para normalizar el padron y dejarlo Limpio y operativo
'''
### COMIENZO A TRABAJAR SOBRE EL PADRON ###
    # Cargo la Informacion del padron
try:
    # Indico que columnas voy a necesitar
    cols = ['GSX', 'NOMBRE', 'Fecha apertura', 'FIN DE CIERRE','ORGANIZACIÓN ', 'DIRECTOR EXPLOTACIÓN', 'DIRECTOR OPERACIONAL', 'DIRECTOR / GERENTE REGIONAL', 'SUB REGION', 'DIRECTOR/ GERENTE TIENDA', 'Provincia Tableau', 'M² SALÓN', 'M² PGC', 'M² PFT', 'M² BAZAR', 'M² Electro', 'M² Textil', 'M² Pls', 'M² GALERIAS', 'PROVINCIA', 'M² Parcking', 'CAJAS', 'COD.POSTAL']
    pad = pd.read_excel('data/padron.xlsx', header=17, usecols=cols)

except Exception as e:
    print('Error a la hora de cargar el Padron. Detalle {e}')

# Estandarizo un poco los nombres de las columnas
pad.columns = pad.columns.str.strip().str.lower().str.replace(' /', '').str.replace('/', '').str.replace('.', ' ')

# Quito nulos
pad = pad.dropna(subset=['nombre', 'organización', 'fecha apertura'])

# Me quedo unicamente con formatos validos
pad = pad[pad['organización'].isin(['HIPERMERCADO', 'MAXI', 'Market', 'Express'])]

# Me quedo con valores que no hayan cerrado
pad = pad[pad['fin de cierre'] == '-']

# Quito columna incompleta
pad = pad.drop(columns={'provincia tableau'})

# Renombro la columna completa de provincias
pad = pad.rename(columns={
    'provincia':'provincia tableau',
    'gsx':'id tienda'
})

# Genero un list aux para bucle abajo
cols = ['m² pgc', 'm² pft', 'm² bazar', 'm² electro', 'm² textil', 'm² pls', 'm² galerias', 'cajas', 'm² parcking']

# Itero sobre cols para estandarizar, rellenar y limpiar nulos
for col in cols:
    pad[col] = pad[col].fillna(0).replace('-', 0).replace('sd', 0).replace('SD', 0).replace('', 0).astype(int)

# Ordeno columnas
pad = pad[['nombre', 'id tienda', 'organización', 'director explotación', 'director operacional', 'director gerente regional', 'sub region', 'director gerente tienda', 'provincia tableau', 'm² salón', 'm² pgc', 'm² pft', 'm² bazar', 'm² electro', 'm² textil', 'm² pls', 'm² galerias', 'cod postal', 'cajas', 'm² parcking']]

# Convierto el codigo postal a String ya que es alfanumerico
pad['cod postal'] = pad['cod postal'].astype(str)

# Genero una nueva columna para identificar la ultima fecha de actualizacion de las tiendas
pad['modificacion'] = datetime.today().strftime('%d/%m/%Y')

# Formateo la fecha y la convierto a String. Como no es util para comprar o utilizar en series de tiempo, sirve igual
pad['modificacion'] = pad['modificacion'].astype('str')

# Reseteo y quito Indice indeseado
pad = pad.reset_index(drop=True)

# Realizo una transformacion para Normalizar valores de las provincias
pad['provincia tableau'] = pad['provincia tableau'].str.strip().str.upper()
pad['provincia tableau'] = np.where(pad['provincia tableau'] == 'NEUQUÉN', 'NEUQUEN', pad['provincia tableau'])


C:\Users\Juan Mera\AppData\Local\Temp\ipykernel_15904\1542635482.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pad[col] = pad[col].fillna(0).replace('-', 0).replace('sd', 0).replace('SD', 0).replace('', 0).astype(int)
C:\Users\Juan Mera\AppData\Local\Temp\ipykernel_15904\1542635482.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pad[col] = pad[col].fillna(0).replace('-', 0).replace('sd', 0).replace('SD', 0).replace('', 0).astype(int)
C:\Users\Juan Mera\AppData\Local\Temp\ipykernel_15904\1542635482.py:40: FutureWarning:

In [18]:
def crear_tabla_si_no_existe(df: pd.DataFrame, table_id: str, project_id: str):
    client = bigquery.Client(project=project_id)

    try:
        client.get_table(table_id)
        print(f"✅ La tabla {table_id} ya existe")
    except NotFound:
        print(f"⚠️ La tabla {table_id} no existe. Creándola...")

        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_EMPTY"
        )

        load_job = client.load_table_from_dataframe(
            dataframe=df,
            destination=table_id,
            job_config=job_config
        )

        load_job.result()
        print(f"✅ Tabla {table_id} creada correctamente.")

In [19]:
def carga_padron(padron, project_id='gcp-ar-cdg-datos-dev', table_id='gcp-ar-cdg-datos-dev.marketshare_project.padron'):
    try:
        # Paso previo: crear tabla si no existe
        crear_tabla_si_no_existe(padron, table_id, project_id)

        # Ahora sí: usar MethodBQ normalmente
        bq_methods = MethodBQ(project=project_id)
        
        bq_methods.upsert_df_to_bigquery(
            df=padron,
            table_id=table_id,
            mode='merge',
            primary_keys=['id tienda']
        )

        print('Exito al cargar la información del padrón a GCP')

    except Exception as e:
        print(f'Error al subir el padrón a GCP: {e}')

carga_padron(padron=pad)

✅ La tabla gcp-ar-cdg-datos-dev.marketshare_project.padron ya existe


c:\Users\Juan Mera\Documents\Python\20_CARREFOUR\progresion\env\Lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:484: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


Exito al cargar la información del padrón a GCP


In [21]:
pad['provincia tableau'].nunique()

24